<a href="https://colab.research.google.com/github/andersonfurtado/AI4WEBDEV/blob/main/ch9/Movie_Recommendation__Chatbot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step-by-Step Python Code

# 1. Install the necessary libraries
!pip install python-telegram-bot scikit-surprise pandas nest_asyncio

# 2. Import the required libraries
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import urllib.request
import zipfile
import os
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import nest_asyncio

# 3. Apply the patch to allow the use of asyncio in Jupyter Notebook
nest_asyncio.apply()

# 4. Download and unzip the MovieLens dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
urllib.request.urlretrieve(url, 'ml-latest-small.zip')

with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall()

movies_file = os.path.join('ml-latest-small', 'movies.csv')
ratings_file = os.path.join('ml-latest-small', 'ratings.csv')

# 5. Load the data
movies = pd.read_csv(movies_file)
ratings = pd.read_csv(ratings_file)

# 6. Prepare the data for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# 7. Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# 8. Train the SVD model
algo = SVD()
algo.fit(trainset)

# 9. Define the movie recommendation function
def get_movie_recommendations(algo, movie_title, movies, ratings, num_recommendations=5):
    movie_id = movies[movies['title'] == movie_title]['movieId'].values[0]
    users_who_rated_movie = ratings[ratings['movieId'] == movie_id]['userId'].unique()
    other_movie_ids = ratings[ratings['userId'].isin(users_who_rated_movie)]['movieId'].unique()
    predicted_ratings = []
    for movie in other_movie_ids:
        if movie != movie_id:
            predicted_ratings.append((movie, algo.predict(uid=0, iid=movie).est))
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n_movies = [movie_id for movie_id, rating in predicted_ratings[:num_recommendations]]
    recommended_movie_titles = movies[movies['movieId'].isin(top_n_movies)]['title'].tolist()
    return recommended_movie_titles

# 10. Set up the Telegram bot
TOKEN = '7282222179:AAHTS7Ye_0WZXhzgxOVvp-jIKAVZ__fhwRA'

async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text('Hello! I am FilmGuideBot. How are you feeling today?')

async def handle_message(update: Update, context: CallbackContext) -> None:
    user_input = update.message.text
    sentiment = analyze_sentiment(user_input)
    entities = extract_entities(user_input)
    await update.message.reply_text(f"Sentiment Analysis: Polarity = {sentiment.polarity}, Subjectivity = {sentiment.subjectivity}")
    await update.message.reply_text(f"Named Entities: {entities}")

    movie_title = user_input
    found_movies = check_movie_title(movie_title, session)
    if found_movies:
        recommendations = get_movie_recommendations_from_db(movie_title)
        if recommendations:
            response = f"Recommendations for '{movie_title}':\n" + "\n".join(recommendations)
        else:
            response = f"No recommendations found for '{movie_title}'."
    else:
        response = f"No movies found with title '{movie_title}'."
    await update.message.reply_text(response)

# 11. Configure and Run the Bot
def main():
    application = Application.builder().token(TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.run_polling()

if __name__ == '__main__':
    main()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.6/631.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357239 sha256=e7572696a4d1db72eee62bb8472808ff7825dd13fff5205a898ef1ce31be7e6e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
